In [4]:
import sys, os, shutil
import glob2
import numpy as np
from tqdm import tqdm_notebook as tqdm
from sklearn.model_selection import train_test_split

# Download file

In [5]:
ori_train_base = 'dogs-vs-cats/origianl/train/'
ori_test_base = 'dogs-vs-cats/origianl/test1/'

base_dir = 'dogs-vs-cats/'

train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

train_cats_dir = os.path.join(train_dir, 'cats')
train_dogs_dir = os.path.join(train_dir, 'dogs')
os.makedirs(train_cats_dir, exist_ok=True)
os.makedirs(train_dogs_dir, exist_ok=True)

val_cats_dir = os.path.join(val_dir, 'cats')
val_dogs_dir = os.path.join(val_dir, 'dogs')
os.makedirs(val_cats_dir, exist_ok=True)
os.makedirs(val_dogs_dir, exist_ok=True)


In [6]:

paths = glob2.glob(ori_train_base + '*.*', norm_paths=True)

test_paths = glob2.glob(ori_test_base + '*.*', norm_paths=True)

cat_paths = [path for path in paths if 'cat.' in path]
dog_paths = [path for path in paths if 'dog.' in path]

print(f'Cat images numbers:{len(cat_paths)}')
print(f'Dog images numbers:{len(dog_paths)}')
print(f'Test images numbers:{len(test_paths)}')

train_cat_paths, val_cat_paths, train_dog_paths, val_dog_paths = train_test_split(cat_paths, dog_paths, test_size=0.2, shuffle=True)
print(f'Cat train : {len(train_cat_paths)}, Cat val : {len(val_cat_paths)}')
print(f'Dog train : {len(train_dog_paths)}, Dog val : {len(val_dog_paths)}')


Cat images numbers:12500
Dog images numbers:12500
Test images numbers:12500
Cat train : 10000, Cat val : 2500
Dog train : 10000, Dog val : 2500


In [7]:
cat_paths[-10:]

['dogs-vs-cats/origianl/train\\cat.9990.jpg',
 'dogs-vs-cats/origianl/train\\cat.9991.jpg',
 'dogs-vs-cats/origianl/train\\cat.9992.jpg',
 'dogs-vs-cats/origianl/train\\cat.9993.jpg',
 'dogs-vs-cats/origianl/train\\cat.9994.jpg',
 'dogs-vs-cats/origianl/train\\cat.9995.jpg',
 'dogs-vs-cats/origianl/train\\cat.9996.jpg',
 'dogs-vs-cats/origianl/train\\cat.9997.jpg',
 'dogs-vs-cats/origianl/train\\cat.9998.jpg',
 'dogs-vs-cats/origianl/train\\cat.9999.jpg']

In [7]:
# for path in tqdm(train_cat_paths):
#     base_name, fname = path.split('\\')
#     src = path
#     dst = os.path.join(train_cats_dir, fname)
#     shutil.copyfile(src, dst)

# for path in tqdm(val_cat_paths):
#     base_name, fname = path.split('\\')
#     src = path
#     dst = os.path.join(val_cats_dir, fname)
#     shutil.copyfile(src, dst)
    
# for path in tqdm(train_dog_paths):
#     base_name, fname = path.split('\\')
#     src = path
#     dst = os.path.join(train_dogs_dir, fname)
#     shutil.copyfile(src, dst)

# for path in tqdm(val_dog_paths):
#     base_name, fname = path.split('\\')
#     src = path
#     dst = os.path.join(val_dogs_dir, fname)
#     shutil.copyfile(src, dst)

# Train model

In [8]:
from keras import layers
from keras import models
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model

In [9]:
from keras.backend.tensorflow_backend import set_session
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
sess = tf.Session(config=config)
set_session(sess)

In [12]:
def build_model():
    model = models.Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation='relu',input_shape=(150, 150, 3)))
    model.add(layers.MaxPool2D((2,2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu',input_shape=(150, 150, 3)))
    model.add(layers.MaxPool2D((2,2)))
    model.add(layers.Conv2D(128, (3, 3), activation='relu',input_shape=(150, 150, 3)))
    model.add(layers.MaxPool2D((2,2)))
    model.add(layers.Conv2D(128, (3, 3), activation='relu',input_shape=(150, 150, 3)))
    model.add(layers.MaxPool2D((2,2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy', optimizer=optimizers.RMSprop(lr=1e-4), metrics=['acc'])
    
    return model

In [13]:
model = build_model()

In [14]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 15, 15, 128)       147584    
__________

In [15]:
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)


train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=100,
    class_mode='binary')

validate_generator = train_datagen.flow_from_directory(
    val_dir,
    target_size=(150, 150),
    batch_size=100,
    class_mode='binary')

test_generator = train_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=125,
    class_mode='binary' , shuffle=False)

Found 24038 images belonging to 2 classes.
Found 9038 images belonging to 2 classes.
Found 12500 images belonging to 1 classes.


In [16]:
from keras import callbacks
import os

In [18]:
filename = "model_{epoch:02d}-{val_acc:.2f}.hdf5"
history = model.fit_generator( train_generator, 
                              steps_per_epoch=100, 
                              epochs=100, 
                              validation_data=validate_generator, 
                              validation_steps=25,
                             verbose=2,
                             callbacks= [callbacks.ModelCheckpoint(os.path.join('checkpoints', filename), monitor='val_loss', verbose=0, save_best_only=False, save_weights_only=False, mode='auto', period=1)])
model.save('cats_and_dogs_baseline.h5')

Epoch 1/100
 - 47s - loss: 0.6069 - acc: 0.6707 - val_loss: 0.5804 - val_acc: 0.6896
Epoch 2/100
 - 43s - loss: 0.5649 - acc: 0.7050 - val_loss: 0.5362 - val_acc: 0.7352
Epoch 3/100
 - 42s - loss: 0.5331 - acc: 0.7332 - val_loss: 0.5475 - val_acc: 0.7067
Epoch 4/100
 - 42s - loss: 0.5230 - acc: 0.7354 - val_loss: 0.5615 - val_acc: 0.7036
Epoch 5/100
 - 42s - loss: 0.4955 - acc: 0.7555 - val_loss: 0.4711 - val_acc: 0.7824
Epoch 6/100
 - 42s - loss: 0.4794 - acc: 0.7704 - val_loss: 0.4571 - val_acc: 0.7892
Epoch 7/100
 - 42s - loss: 0.4650 - acc: 0.7760 - val_loss: 0.4521 - val_acc: 0.7961
Epoch 8/100
 - 42s - loss: 0.4478 - acc: 0.7908 - val_loss: 0.4225 - val_acc: 0.8084
Epoch 9/100
 - 42s - loss: 0.4397 - acc: 0.7963 - val_loss: 0.4306 - val_acc: 0.7952
Epoch 10/100
 - 42s - loss: 0.4215 - acc: 0.8081 - val_loss: 0.4277 - val_acc: 0.8072
Epoch 11/100
 - 42s - loss: 0.4132 - acc: 0.8112 - val_loss: 0.4081 - val_acc: 0.8208
Epoch 12/100
 - 42s - loss: 0.4008 - acc: 0.8200 - val_loss: 0.

Epoch 97/100
 - 48s - loss: 0.0216 - acc: 0.9939 - val_loss: 0.0680 - val_acc: 0.9844
Epoch 98/100
 - 44s - loss: 0.0237 - acc: 0.9935 - val_loss: 0.0629 - val_acc: 0.9852
Epoch 99/100
 - 45s - loss: 0.0247 - acc: 0.9934 - val_loss: 0.0611 - val_acc: 0.9836
Epoch 100/100
 - 45s - loss: 0.0233 - acc: 0.9933 - val_loss: 0.0788 - val_acc: 0.9832


In [13]:
model = load_model('cats_and_dogs_baseline.h5')

In [19]:
test_generator.reset()
labels = model.predict_generator(test_generator, steps=100).ravel()

In [20]:
import re

In [21]:
_ids = []
for fname in test_generator.filenames:
    _ids.append(re.findall(r'\\(\d+).jpg', fname)[0])
    

In [22]:
import pandas as pd

In [23]:
len(labels), len(result)

NameError: name 'result' is not defined

In [ ]:
len(labels[0]), len(_ids)

In [ ]:
df = pd.DataFrame.from_dict({'id': _ids, 'label':labels} )

In [ ]:
df.to_csv('test_result.csv', index=False)

In [ ]:
!kaggle competitions submit -c dogs-vs-cats-redux-kernels-edition -f test_result.csv -m "Testing submissiion"